In [1]:
#%pip install pandas
#%pip install numpy
#%pip install matplotlib
#%pip install openpyxl
%pip install plotly
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import exp

from math import exp
class SingleMachine:
    def __init__(self, n = 1 , p = [], d = [], r = [], start=0):
        """n: number of jobs
        #M: number of machines
        #J: number of jobs
        #p: processing time
        #d: due date
        #r: release date
        #start: start time"""
        self.n = n
        self.J = list(range(n))
        self.p = p
        self.d = d
        self.r = r
        self.start = start
    def process(self):
        #S: start time for job j
        #C: completion time for job j
        #L: lateness for job j
        #T: tardiness for job j
        #E: earliness for job j
        self.S = [0]*self.n
        self.C = [0]*self.n
        self.L = [0]*self.n
        self.T = [0]*self.n
        self.E = [0]*self.n
        self.C[-1] = self.start
        for j in range(self.n):
            self.S[j] = max(self.r[j], self.C[j-1])
            self.C[j] = self.S[j] + self.p[j]
            self.L[j] = self.C[j] - self.d[j]
            self.T[j] = max(0, self.L[j])
            self.E[j] = max(0, -self.L[j])
    def FCFS(self):
        self.J = sorted(self.J)
        self.process()
        return self.J
    def LCFS(self):
        self.J = sorted(self.J, reverse=True)
        self.process()
        return self.J
    FIFO = FCFS
    LIFO = LCFS
    def SPT(self):
        if self.r == [] or len(set(self.r)) == 1:
            self.J = [self.J for _, self.J in sorted(zip(self.p, self.J))]
            self.d = [self.d for _, self.d in sorted(zip(self.p, self.d))]
            self.p = sorted(self.p)
            self.process()
            return self.J
        else:
            #Sort the jobs by release date and processing time
            self.J = [x for _, x in sorted(zip(zip(self.r, self.p), self.J))]
            self.d = [x for _, x in sorted(zip(zip(self.r, self.p), self.d))]
            self.p = [x for _, x in sorted(zip(zip(self.r, self.p), self.p))]
            self.r = sorted(self.r)
        self.process()
        return self.J
    def LPT(self):
        self.J = [self.J for _, self.J in sorted(zip(self.p, self.J), reverse=True)]
        self.d = [self.d for _, self.d in sorted(zip(self.p, self.d), reverse=True)]
        self.p = sorted(self.p, reverse=True)
        self.process()
        return self.J
    def EDD(self):
        self.J = [self.J for _, self.J in sorted(zip(self.d, self.J))]
        self.p = [self.p for _, self.p in sorted(zip(self.d, self.p))]
        self.d = sorted(self.d)
        self.process()
        return self.J
    def LDD(self):
        self.J = [self.J for _, self.J in sorted(zip(self.d, self.J), reverse=True)]
        self.p = [self.p for _, self.p in sorted(zip(self.d, self.p), reverse=True)]
        self.d = sorted(self.d, reverse=True)
        self.process()
        return self.J
    def CR(self, check_time=False):
        if check_time == False:
            SyntaxError("check_time is not defined, Start time is used instead")
            check_time = self.start
        t = check_time
        CR = [0]*self.n
        for j in self.J:
            CR[j] = (self.d[j]-t)/self.p[j]
        self.J = [self.J for _, self.J in sorted(zip(CR, self.J))]
        self.p = [self.p for _, self.p in sorted(zip(CR, self.p))]
        self.d = [self.d for _, self.d in sorted(zip(CR, self.d))]
        self.process()
        return self.J
    CriticalRatio = CR
    def MinimumSlack(self, check_time=False):
        if check_time == False:
            SyntaxError("check_time is not defined, Start time is used instead")
            check_time = self.start
        t = check_time
        MS = [0]*self.n
        for j in self.J:
            MS[j] = max(0,self.d[j]-t-self.p[j])
        self.J = [self.J for _, self.J in sorted(zip(MS, self.J))]
        self.p = [self.p for _, self.p in sorted(zip(MS, self.p))]
        self.d = [self.d for _, self.d in sorted(zip(MS, self.d))]
        self.process()
        return self.J
    MinSlack = MinimumSlack
    def MaximumSlack(self, check_time=False):
        if check_time == False:
            SyntaxError("check_time is not defined, Start time is used instead")
            check_time = self.start
        t = check_time
        MS = [0]*self.n
        for j in self.J:
            MS[j] = max(0,self.d[j]-t-self.p[j])
        self.J = [self.J for _, self.J in sorted(zip(MS, self.J), reverse=True)]
        self.p = [self.p for _, self.p in sorted(zip(MS, self.p), reverse=True)]
        self.d = [self.d for _, self.d in sorted(zip(MS, self.d), reverse=True)]
        self.process()
        return self.J
    MaxSlack = MaximumSlack
    def ATC(self, check_time=False, K =False):
        if check_time == False:
            SyntaxError("check_time is not defined, Start time is used instead")
            check_time = self.start
        if K == False:
            SyntaxError("K is not defined, K = 1 is used instead")
            K = 1
        t = check_time
        ATC = [0]*self.n
        P = sum(self.p)/self.n
        for j in self.J:
            ATC[j] = (1/self.n)/(self.p[j])*exp(-max(self.d[j]-t-self.p[j], 0)/(K*P))
        self.J = [self.J for _, self.J in sorted(zip(ATC, self.J), reverse=True)]
        self.p = [self.p for _, self.p in sorted(zip(ATC, self.p), reverse=True)]
        self.d = [self.d for _, self.d in sorted(zip(ATC, self.d), reverse=True)]
        self.process()
        return self.J
    def CommonDueDate(self):
        if len(set(self.d)) != 1:
            SyntaxError("Not All jobs have the same due date")
        #Step 0: Rank the jobs in SPT order
        self.SPT()
        #Step 1: Create two sets A and B
        A = []
        B = []
        #Step 2: Compute Cmax = \sum_{j=1}^{n} p_j, i = n, R = Cmax-d
        n = self.n
        Cmax = sum(self.p)
        i = n
        R = Cmax - self.d[0]
        L = self.d[0]
        #Step 3: If R > L, then add job i to set A and go to step 4
        while i > 0:
            if R >= L:
                A.append(i)
                i -= 1
                R = R-self.p[i]
            else:
                B.append(i)
                i -= 1
                L = L-self.p[i] 
        Order = B + list(reversed(A))
        Lab = [0]*n
        for j in range(n):
            Lab[j] = self.J[Order[j]-1]
        self.J = Lab
        self.process()
        return self.J
    def DiferentDueDates(self, groups: int,  P1: float = 1, P2: float = 1, P3: float = 1):
        if len(set(self.d)) == 1:
            SyntaxError("All jobs have the same due date, Common Due Date is recomended instead")
        #Step 1: Rank the jobs in SPT order
        self.SPT()
        #Step 2: Set N[0] = 0, N[j] = sum_{k=1}^j n_k, j = 1,2,...,n
        n = list(range(self.n))
        N = [0]*self.n
        N[0] = 0
        for j in range(1,n):
            sum(n[:j])
DatosTaller = pd.read_excel('DatosTaller1.xlsx', sheet_name='Sheet1')
# drop nan columns and rows
DatosTaller = DatosTaller.dropna(axis=1, how='all')
DatosTaller = DatosTaller.dropna(axis=0, how='all')
display(DatosTaller)
display(DatosTaller.describe())
keys = DatosTaller.keys()
print(keys)
for i in keys:
    display(DatosTaller[i].value_counts())
    

In [ ]:
def get_p(row):
    """
    function that evaluates when a row and creates a column based on the
    value of 'Tipo' where:
    if the value is 'Figurado' it returns 'KILOS'/ 2600,
    if the value is 'Mallas' it returns 'KILOS'/ 800, and
    if the value is 'Materia Prima' it returns 0
    """
    if row['Tipo'] == 'Figurado':
        return row['KILOS']/2600
    elif row['Tipo'] == 'Mallas':
        return row['KILOS']/800
    else:
        return 0
def add_nhours(row, n):
    """
    function that adds n number of hours to a date on a row
    to the column 'FECHA'
    """
    # get the date on the row
    date = row['FECHA']
    # add n hours to the date
    date = date + pd.Timedelta(hours=n)
    return date
def get_weekday(row):
    """
    function that takes a row and returns the day of the week
    """
    # get the date on the row
    date = row['FECHA']
    # get the day of the week
    weekday = date.day_name()
    return weekday

def get_r(row):
    """
    function that takes a row and returns the hours since the minimum date
    on the column 'FECHA' and ignores the time between Saturday and Sunday
    """
    # get the minimum date
    min_date = DatosTaller['FECHA'].min()
    # get the difference between the row date and the minimum date
    diff = row['FECHA'] - min_date
    # get the number of days since the minimum date
    days = diff.days
    # get the number of weeks since the minimum date
    weeks = days // 7
    # get the number of days since the minimum date ignoring the weekends
    days = days - 2*weeks
    # get the number of hours since the minimum date ignoring the weekends
    hours = days*24
    return hours
def get_d(row, n):
    """
    function that takes a row and returns r_{j} and adds n hours
    """
    # get the number of hours since the minimum date ignoring the weekends
    r = row['r']
    # add n hours to r_{j}
    r = r + n
    return r
# use the function add_nhours to add 15 hours to the column 'FECHA'
DatosTaller['FECHA'] = DatosTaller.apply(add_nhours, axis=1, args=(15,))
# create a new column 'p_{j}' and use the function get_p to fill it
DatosTaller['p'] = DatosTaller.apply(get_p, axis=1)
# create a new column 'weekday' and use the function get_weekday to fill it
DatosTaller['weekday'] = DatosTaller.apply(get_weekday, axis=1)
# create a new column 'r_{j}' and use the function get_r to fill it
DatosTaller['r'] = DatosTaller.apply(get_r, axis=1)
# create a new column 'd_{j}' and use the function get_d to fill it
DatosTaller['d'] = DatosTaller.apply(get_d, axis=1, args=(24,))
display(DatosTaller)

n = len(DatosTaller)
p = DatosTaller['p'].tolist()
d = DatosTaller['d'].tolist()
r = DatosTaller['r'].tolist()
SM = SingleMachine(n=n, p=p, d=d, r=r)
SM.SPT()

resultado = pd.DataFrame({'Job': SM.J,
                          'p': SM.p,
                          'd': SM.d,
                          'r': SM.r,
                          'S': SM.S,
                          'C': SM.C,
                          'L': SM.L,
                          'T': SM.T,
                          'E': SM.E})
display(resultado)
resultado.to_excel('resultado.xlsx')

,Nº REMISION,FECHA,Columna1,UND,KILOS,Tipo,DESCRIPCION,P - Horas,D -Dias,Columna2,Columna3,Columna5,Columna7,S,C,p,weekday,r,d
0,1000016975,2011-01-03 15:00:00,2011-01-03 15:00:00,648.0,648.0,Materia Prima,MAQUILA VARILLA RECTA 60 MIL N.6 X 12 MTS,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 15:00:00.000,2011-01-03 15:00:00.000,0.000000,Monday,0,24
1,1000016975,2011-01-03 15:00:00,2011-01-03 15:00:00,1175.0,1175.0,Materia Prima,MAQUILA VARILLA RECTA 60 MIL N.7 X 12 MTS,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 15:00:00.000,2011-01-03 15:00:00.000,0.000000,Monday,0,24
2,1000016982,2011-01-03 15:00:00,2011-01-03 15:00:00,108.0,108.0,Materia Prima,MAQUILA VARILLA RECTA 60 MIL N.4 X 12 MTS,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 15:00:00.000,2011-01-03 15:00:00.000,0.000000,Monday,0,24
3,1000016989,2011-01-03 15:00:00,2011-01-03 15:00:00,100.0,100.0,Materia Prima,MAQUILA ALAMBRE RECOCIDO CAL 18,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 15:00:00.000,2011-01-03 15:00:00.000,0.000000,Monday,0,24
4,1000016990,2011-01-03 15:00:00,2011-01-03 15:00:00,250.0,250.0,Materia Prima,MAQUILA ALAMBRE RECOCIDO CAL 18,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 15:00:00.000,2011-01-03 15:00:00.000,0.000000,Monday,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,1000018120,2011-02-15 15:00:00,2011-02-15 15:00:00,2319.9,2319.9,Figurado,MAQUILA ACERO FIGURADO 4,00:53:32.169000,2011-02-16 15:00:00,2011-02-15 15:53:32.169,True,5,2235,2011-03-18 08:20:37.888,2011-03-18 09:14:10.057,0.892269,Tuesday,744,768
2278,1000018122,2011-02-15 15:00:00,2011-02-15 15:00:00,5514.1,5514.1,Figurado,MAQUILA ACERO FIGURADO 4,02:07:14.908000,2011-02-16 15:00:00,2011-02-15 17:07:14.908,True,4,2235,2011-03-18 09:14:10.057,2011-03-18 11:21:24.965,2.120808,Tuesday,744,768
2279,1000018143,2011-02-15 15:00:00,2011-02-15 15:00:00,10464.0,10464.0,Figurado,MAQUILA ACERO FIGURADO 8,04:01:28.615000,2011-02-16 15:00:00,2011-02-15 19:01:28.615,True,3,2235,2011-03-18 11:21:24.965,2011-03-18 15:22:53.580,4.024615,Tuesday,744,768
2280,1000018117,2011-02-15 15:00:00,2011-02-15 15:00:00,10705.8,10705.8,Figurado,MAQUILA ACERO FIGURADO 7,04:07:03.415000,2011-02-16 15:00:00,2011-02-15 19:07:03.415,True,2,2235,2011-03-18 15:22:53.580,2011-03-18 19:29:56.995,4.117615,Tuesday,744,768


,Job,p,d,r,S,C,L,T,E
0,0,0.000000,24,0,0.000000,0.000000,-24.000000,0.000000,24.0
1,1,0.000000,24,0,0.000000,0.000000,-24.000000,0.000000,24.0
2,2,0.000000,24,0,0.000000,0.000000,-24.000000,0.000000,24.0
3,3,0.000000,24,0,0.000000,0.000000,-24.000000,0.000000,24.0
4,4,0.000000,24,0,0.000000,0.000000,-24.000000,0.000000,24.0
...,...,...,...,...,...,...,...,...,...
2277,2277,0.892269,768,744,1769.343858,1770.236127,1002.236127,1002.236127,0.0
2278,2278,2.120808,768,744,1770.236127,1772.356935,1004.356935,1004.356935,0.0
2279,2279,4.024615,768,744,1772.356935,1776.381550,1008.381550,1008.381550,0.0
2280,2280,4.117615,768,744,1776.381550,1780.499165,1012.499165,1012.499165,0.0
